In [1]:
# sql course website
# https://lagunita.stanford.edu/courses/DB/SQL/SelfPaced/courseware/ch-sql/seq-vid-table_variables_and_set_operators/?child=first

import sqlite3

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except IOError as e:
        print(e)

    return None


def create_table(conn, create_table_sql):
    #create a table from the create_table_sql statement
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except IOError as e:
        print(e)

In [2]:
create_college_table = """CREATE TABLE IF NOT EXISTS College (
                            cName text PRIMARY KEY,
                            state text NOT NULL,
                            enrollment integer NOT NULL);"""

create_student_table = """CREATE TABLE IF NOT EXISTS Student (
                            sID integer PRIMARY KEY,
                            sName text NOT NULL,
                            GPA real NOT NULL,
                            sizeHS integer NOT NULL);"""

create_apply_table = """CREATE TABLE IF NOT EXISTS Apply (
                            sID integer NOT NULL,
                            cName text NOT NULL,
                            major text NOT NULL,
                            decision text NOT NULL,
                            FOREIGN KEY (sID) REFERENCES Student (sID),
                            FOREIGN KEY (cName) REFERENCES College (cName));"""

In [3]:
database = "C:\Users\wdew\Desktop\jupyter\SQL\simple_college_admissions.db"

# create a database connection
conn = create_connection(database)
if conn is not None:
    create_table(conn, create_college_table)
    create_table(conn, create_student_table)
    create_table(conn, create_apply_table)
    conn.close()
else:
    print("Error! cannot create the database connection.")

In [4]:
def create_college(conn, college):
    """
    Create a new college into the College table
    :param conn:
    :param college:
    """
    sql = ''' INSERT INTO College(cName, state, enrollment)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, college)

def create_student(conn, student):
    """
    Create a new student into the Student table
    :param conn:
    :param student:
    """
    sql = ''' INSERT INTO Student(sID, sName, GPA, sizeHS)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, student)

def create_apply(conn, apply_1):
    """
    Create a new application into the Apply table
    :param conn:
    :param apply_1:
    """
    sql = ''' INSERT INTO Apply(sID, cName, major, decision)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, apply_1)

In [5]:
college_dict = {1: ('Stanford', 'CA', 15000),
                2: ('Berkeley', 'CA', 36000),
                3: ('MIT', 'MA', 10000),
                4: ('Cornell', 'NY', 21000)}

student_dict = {1: (123, 'Amy', 3.9, 1000),
                2: (234, 'Bob', 3.6, 1500),
                3: (345, 'Craig', 3.5, 500),
                4: (456, 'Doris', 3.9, 1000),
                5: (567, 'Edward', 2.9, 2000),
                6: (678, 'Fay', 3.8, 200),
                7: (789, 'Gary', 3.4, 800),
                8: (987, 'Helen', 3.7, 800),
                9: (876, 'Irene', 3.9, 400),
                10: (765, 'Jay', 2.9, 1500),
                11: (654, 'Amy', 3.9, 1000),
                12: (543, 'Craig', 3.4, 2000)}

application_dict = {1: (123, 'Stanford', 'CS', 'Y'),
                    2: (123, 'Stanford', 'EE', 'N'),
                    3: (123, 'Berkeley', 'CS', 'Y'),
                    4: (123, 'Cornell', 'EE', 'Y'),
                    5: (234, 'Berkeley', 'biology', 'N'),
                    6: (345, 'MIT', 'bioengineering', 'Y'),
                    7: (345, 'Cornell', 'bioengineering', 'N'),
                    8: (345, 'Cornell', 'CS', 'Y'),
                    9: (345, 'Cornell', 'EE', 'N'),
                    10: (678, 'Stanford', 'history', 'Y'),
                    11: (987, 'Stanford', 'CS', 'Y'),
                    12: (987, 'Berkeley', 'CS', 'Y'),
                    13: (876, 'Stanford', 'CS', 'N'),
                    14: (876, 'MIT', 'biology', 'Y'),
                    15: (876, 'MIT', 'marine biology', 'N'),
                    16: (765, 'Stanford', 'history', 'Y'),
                    17: (765, 'Cornell', 'history', 'N'),
                    18: (765, 'Cornell', 'psychology', 'Y'),
                    19: (543, 'MIT', 'CS', 'N')}

In [6]:
conn = create_connection(database)
with conn:
    for value in college_dict.values():
        create_college(conn, value)
    for value in student_dict.values():
         create_student(conn, value)
    for value in application_dict.values():
        create_apply(conn, value)
conn.close()

In [7]:
def select(conn, query):
    """
    Query tasks by priority
    :param conn: the Connection object
    :param priority:
    :return:
    """
    cur = conn.cursor()
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)    

In [8]:
conn = create_connection(database)
query = """select sID, sName from Student where GPA > 3.6;"""
select(conn, query)
conn.close()

(123, u'Amy')
(456, u'Doris')
(654, u'Amy')
(678, u'Fay')
(876, u'Irene')
(987, u'Helen')


In [9]:
conn = create_connection(database)
query = """select distinct sName, major
            from Student, Apply
            where Student.sID = Apply.sID;"""
select(conn, query)
conn.close()

(u'Amy', u'CS')
(u'Amy', u'EE')
(u'Bob', u'biology')
(u'Craig', u'bioengineering')
(u'Craig', u'CS')
(u'Craig', u'EE')
(u'Fay', u'history')
(u'Helen', u'CS')
(u'Irene', u'CS')
(u'Irene', u'biology')
(u'Irene', u'marine biology')
(u'Jay', u'history')
(u'Jay', u'psychology')


In [10]:
conn = create_connection(database)
query = """select sName, GPA, decision
            from Student, Apply
            where Student.sID = Apply.sID
            and sizeHS < 1000 and major = 'CS' and cName = 'Stanford';"""
select(conn, query)
conn.close()

(u'Helen', 3.7, u'Y')
(u'Irene', 3.9, u'N')


In [11]:
conn = create_connection(database)
query = """select distinct College.cName
            from College, Apply
            where College.cName = Apply.cName
            and enrollment > 20000 and major = 'CS';"""
select(conn, query)
conn.close()

(u'Berkeley',)
(u'Cornell',)


In [12]:
conn = create_connection(database)
query = """select Student.sID, sName, GPA, Apply.cName, enrollment
            from Student, College, Apply
            where Apply.sID = Student.sID
            and Apply.cName = College.cName
            order by GPA desc, enrollment;"""
select(conn, query)
conn.close()

(876, u'Irene', 3.9, u'MIT', 10000)
(876, u'Irene', 3.9, u'MIT', 10000)
(123, u'Amy', 3.9, u'Stanford', 15000)
(123, u'Amy', 3.9, u'Stanford', 15000)
(876, u'Irene', 3.9, u'Stanford', 15000)
(123, u'Amy', 3.9, u'Cornell', 21000)
(123, u'Amy', 3.9, u'Berkeley', 36000)
(678, u'Fay', 3.8, u'Stanford', 15000)
(987, u'Helen', 3.7, u'Stanford', 15000)
(987, u'Helen', 3.7, u'Berkeley', 36000)
(234, u'Bob', 3.6, u'Berkeley', 36000)
(345, u'Craig', 3.5, u'MIT', 10000)
(345, u'Craig', 3.5, u'Cornell', 21000)
(345, u'Craig', 3.5, u'Cornell', 21000)
(345, u'Craig', 3.5, u'Cornell', 21000)
(543, u'Craig', 3.4, u'MIT', 10000)
(765, u'Jay', 2.9, u'Stanford', 15000)
(765, u'Jay', 2.9, u'Cornell', 21000)
(765, u'Jay', 2.9, u'Cornell', 21000)


In [13]:
conn = create_connection(database)
query = """select sID, major
            from Apply
            where major like '%bio%';"""
select(conn, query)
conn.close()

(234, u'biology')
(345, u'bioengineering')
(345, u'bioengineering')
(876, u'biology')
(876, u'marine biology')


In [14]:
conn = create_connection(database)
query = """select *
            from Apply
            where major like '%bio%';"""
select(conn, query)
conn.close()

(234, u'Berkeley', u'biology', u'N')
(345, u'MIT', u'bioengineering', u'Y')
(345, u'Cornell', u'bioengineering', u'N')
(876, u'MIT', u'biology', u'Y')
(876, u'MIT', u'marine biology', u'N')


In [15]:
conn = create_connection(database)
query = """select sID, sName, GPA, sizeHS, GPA*(sizeHS/1000.0) as scaledGPA
            from Student;"""
select(conn, query)
conn.close()

(123, u'Amy', 3.9, 1000, 3.9)
(234, u'Bob', 3.6, 1500, 5.4)
(345, u'Craig', 3.5, 500, 1.75)
(456, u'Doris', 3.9, 1000, 3.9)
(543, u'Craig', 3.4, 2000, 6.8)
(567, u'Edward', 2.9, 2000, 5.8)
(654, u'Amy', 3.9, 1000, 3.9)
(678, u'Fay', 3.8, 200, 0.76)
(765, u'Jay', 2.9, 1500, 4.35)
(789, u'Gary', 3.4, 800, 2.72)
(876, u'Irene', 3.9, 400, 1.56)
(987, u'Helen', 3.7, 800, 2.9600000000000004)


In [16]:
conn = create_connection(database)
query = """select s.sID, sName, GPA, a.cName, enrollment
            from Student s, College c , Apply a
            where a.sID = s.sID and a.cName = c.cName;"""
select(conn, query)
conn.close()

(123, u'Amy', 3.9, u'Stanford', 15000)
(123, u'Amy', 3.9, u'Stanford', 15000)
(123, u'Amy', 3.9, u'Berkeley', 36000)
(123, u'Amy', 3.9, u'Cornell', 21000)
(234, u'Bob', 3.6, u'Berkeley', 36000)
(345, u'Craig', 3.5, u'MIT', 10000)
(345, u'Craig', 3.5, u'Cornell', 21000)
(345, u'Craig', 3.5, u'Cornell', 21000)
(345, u'Craig', 3.5, u'Cornell', 21000)
(678, u'Fay', 3.8, u'Stanford', 15000)
(987, u'Helen', 3.7, u'Stanford', 15000)
(987, u'Helen', 3.7, u'Berkeley', 36000)
(876, u'Irene', 3.9, u'Stanford', 15000)
(876, u'Irene', 3.9, u'MIT', 10000)
(876, u'Irene', 3.9, u'MIT', 10000)
(765, u'Jay', 2.9, u'Stanford', 15000)
(765, u'Jay', 2.9, u'Cornell', 21000)
(765, u'Jay', 2.9, u'Cornell', 21000)
(543, u'Craig', 3.4, u'MIT', 10000)


In [17]:
conn = create_connection(database)
query = """select s1.sID, s1.sName, s1.GPA, s2.sID, s2.sName, s2.GPA
            from Student s1, Student s2
            where s1.GPA = s2.GPA AND s1.sID < s2.sID;"""
select(conn, query)
conn.close()

(123, u'Amy', 3.9, 654, u'Amy', 3.9)
(123, u'Amy', 3.9, 456, u'Doris', 3.9)
(123, u'Amy', 3.9, 876, u'Irene', 3.9)
(456, u'Doris', 3.9, 654, u'Amy', 3.9)
(456, u'Doris', 3.9, 876, u'Irene', 3.9)
(543, u'Craig', 3.4, 789, u'Gary', 3.4)
(567, u'Edward', 2.9, 765, u'Jay', 2.9)
(654, u'Amy', 3.9, 876, u'Irene', 3.9)


In [18]:
conn = create_connection(database)
query = """select cName as name from College
            union all
            select sName as name from Student
            order by name;"""
select(conn, query)
conn.close()

(u'Amy',)
(u'Amy',)
(u'Berkeley',)
(u'Bob',)
(u'Cornell',)
(u'Craig',)
(u'Craig',)
(u'Doris',)
(u'Edward',)
(u'Fay',)
(u'Gary',)
(u'Helen',)
(u'Irene',)
(u'Jay',)
(u'MIT',)
(u'Stanford',)


In [19]:
conn = create_connection(database)
query = """select sID from Apply where major = 'CS'
            intersect
            select sID from Apply where major = 'EE';"""
select(conn, query)
conn.close()

(123,)
(345,)


In [20]:
conn = create_connection(database)
query = """select sID from Apply where major = 'CS'
            except
            select sID from Apply where major = 'EE';"""
select(conn, query)
conn.close()

(543,)
(876,)
(987,)


## Subqueries in WHERE clause

In [21]:
# IDs and names of students who have applied to major in CS at some college
conn = create_connection(database)
query = """ select sID, sName
            from Student
            where sID in (select sID from Apply where major = 'CS');"""
select(conn, query)
conn.close()

(123, u'Amy')
(345, u'Craig')
(543, u'Craig')
(876, u'Irene')
(987, u'Helen')


In [22]:
# Duplicates are important: average GPA of CS applicants
conn = create_connection(database)
query = """ select GPA
            from Student
            where sID in (select sID from Apply where major = 'CS');"""
select(conn, query)
conn.close()

(3.9,)
(3.5,)
(3.4,)
(3.9,)
(3.7,)


In [23]:
# Students who have applied to major in CS but have not applied to major in EE
conn = create_connection(database)
query = """ select sID, sName
            from Student
            where sID in (select sID from Apply where major = 'CS')
            and sID not in (select sID from apply where major = 'EE');"""
select(conn, query)
conn.close()

(543, u'Craig')
(876, u'Irene')
(987, u'Helen')


In [24]:
# all colleges such that some other college is in the same state
conn = create_connection(database)
query = """ select cName, state
            from College C1
            where exists (select * from College C2 where C1.state = C2.state
                            and C1.cName <> C2.cName);"""
select(conn, query)
conn.close()

(u'Stanford', u'CA')
(u'Berkeley', u'CA')


In [25]:
# college with highest enrollment
conn = create_connection(database)
query = """ select cName
            from College c1
            where not exists (select * from College c2
            where c2.enrollment > c1.enrollment);"""
select(conn, query)
conn.close()

(u'Berkeley',)


In [26]:
# student with highest GPA
conn = create_connection(database)
query = """ select sName, GPA
            from Student S1
            where not exists (select * from Student s2 
            where s2.GPA > s1.GPA);"""
select(conn, query)
conn.close()

(u'Amy', 3.9)
(u'Doris', 3.9)
(u'Amy', 3.9)
(u'Irene', 3.9)


In [27]:
conn = create_connection(database)
query = """ select sName, GPA
            from Student
            where GPA >= all (select GPA from Student);"""
select(conn, query)
conn.close()

OperationalError: near "all": syntax error

In [28]:
conn = create_connection(database)
query = """ select sName, GPA
            from Student S1
            where GPA > all (select GPA from Student S2 where S2.sID <> s1.sID);"""
select(conn, query)
conn.close()

OperationalError: near "all": syntax error

In [29]:
# college with highest enrollment with all
conn = create_connection(database)
query = """ select cName
            from College C1
            where enrollment > all (select enrollment from College C2
                                    where C2.cName <> C1.cName);"""
select(conn, query)
conn.close()

OperationalError: near "all": syntax error

In [30]:
# students not from the smallest high school
conn = create_connection(database)
query = """ select sID, sName, sizeHS
            from Student
            where sizeHS > any (select sizeHS from Student);"""
select(conn, query)
conn.close()

OperationalError: near "select": syntax error

In [31]:
# Students who have applied to CS but not EE
conn = create_connection(database)
query = """ select sID, sName
            from Student
            where sID = any (select sID from Apply where major = 'CS')
            and not sID = any (select sID from apply where major = 'EE');"""
select(conn, query)
conn.close()

OperationalError: near "select": syntax error

## Subqueries in FROM and SELECT

In [32]:
# Students whose scaled GPA changes GPA by more than 1
conn = create_connection(database)
query = """ select *
            from (select sID, sName, GPA, GPA*(sizeHS/1000.0) as scaledGPA
                    from Student) G
            where abs(G.scaledGPA - GPA) > 1.0;"""
select(conn, query)
conn.close()

(234, u'Bob', 3.6, 5.4)
(345, u'Craig', 3.5, 1.75)
(543, u'Craig', 3.4, 6.8)
(567, u'Edward', 2.9, 5.8)
(678, u'Fay', 3.8, 0.76)
(765, u'Jay', 2.9, 4.35)
(876, u'Irene', 3.9, 1.56)


In [33]:
# colleges paired with the highest GPA of their applicants
conn = create_connection(database)
query = """ select cName, state, (select distinct GPA
                                    from Apply, Student
                                    where College.cName = Apply.cName
                                    and Apply.sID = Student.sID
                                    and GPA >= all 
                                    (select GPA from Student, Apply
                                    where Student.sID = Apply.sID
                                    and Apply.cName = College.cName)) as GPA
            from College;"""
select(conn, query)
conn.close()

OperationalError: near "all": syntax error

## The JOIN Family of Operators

In [ ]:
# colleges paired with the highest GPA of their applicants
conn = create_connection(database)
query = """ ;"""
select(conn, query)
conn.close()